In [22]:
import pandas as pd
import numpy as np
import re
# import nltk
import string
import random
import seaborn as sns

# from sklearn.feature_extraction.text import TfidfVectorizer
# from sklearn.preprocessing import MultiLabelBinarizer, LabelBinarizer
# from sklearn.model_selection import train_test_split
# from sklearn import metrics
# from sklearn.metrics import classification_report, accuracy_score, hamming_loss, f1_score
# from sklearn.linear_model import LogisticRegression
# from sklearn.svm import LinearSVC

# from nltk.corpus import stopwords
# from nltk.stem import WordNetLemmatizer

# nltk.download('stopwords')
# nltk.download('wordnet')
# nltk.download('omw-1.4')

In [23]:
quest = pd.read_csv('data/questions.csv') 
questPost = pd.read_csv("questionposts_cleaned.csv", on_bad_lines='skip') 
questFull = pd.merge(quest, questPost, on = "QuestionUno") 
questFull 

,Id_x,StateAbbr_x,QuestionUno,CategoryUno,Category,SubcategoryUno,Subcategory,AskedByClientUno,AskedOnUtc,TakenByAttorneyUno,TakenOnUtc,ClosedByAttorneyUno,ClosedOnUtc,LegalDeadline,Id_y,StateAbbr_y,PostText,CreatedUtc
0,1,NC,BB53B2A4-7C1B-45C3-9693-62047889317D,C932D197-7F43-4BCF-BE0B-FE5F5DBCB9EF,Family and Children,E9F87919-0CD5-4D9D-AAC6-3CBC49132F1D,Family/Divorce/Custody/Visitation/Child Support,9C35423C-914A-42BC-8EBC-A370E76E2178,2019-06-19 22:10:06,NaN,NaN,NaN,2019-08-13 15:09:06,2019-07-08 00:00:00,215865,NC,"""I called the clerks office, they keep saying ...","""2019-06-19 22:10:36"""
1,2,NC,0C49F841-2782-4313-961E-7882B96D0854,C8E1275A-6C42-42E4-9B7D-231BAC438FC9,Individual Rights,73A59926-BA0B-44EC-ABF3-F4C9181EEA83,Civil/Constitutional Rights,DF636AEE-7D08-4F90-9A5C-0A2A726C6B59,2019-06-19 23:34:36,NaN,NaN,NaN,2019-08-13 15:09:14,NaN,215866,NC,"""I rent a LIHTC apt. with lease rule; Section ...","""2019-06-19 23:35:36"""
2,3,NC,BFC28E08-B6B1-4185-A91A-89DF90816BE4,C8E1275A-6C42-42E4-9B7D-231BAC438FC9,Individual Rights,73A59926-BA0B-44EC-ABF3-F4C9181EEA83,Civil/Constitutional Rights,DF636AEE-7D08-4F90-9A5C-0A2A726C6B59,2019-06-19 23:57:18,FA34142B-1575-4720-981C-2D28C3560137,2019-06-22 02:37:16,FA34142B-1575-4720-981C-2D28C3560137,2019-06-24 19:02:35,NaN,215867,NC,"""My apt. leases states no trash or trash can i...","""2019-06-19 23:57:36"""
3,3,NC,BFC28E08-B6B1-4185-A91A-89DF90816BE4,C8E1275A-6C42-42E4-9B7D-231BAC438FC9,Individual Rights,73A59926-BA0B-44EC-ABF3-F4C9181EEA83,Civil/Constitutional Rights,DF636AEE-7D08-4F90-9A5C-0A2A726C6B59,2019-06-19 23:57:18,FA34142B-1575-4720-981C-2D28C3560137,2019-06-22 02:37:16,FA34142B-1575-4720-981C-2D28C3560137,2019-06-24 19:02:35,NaN,215868,NC,"""Mr. ###:Thank you for your question; you are ...","""2019-06-22 02:49:09"""
4,4,NC,45B24109-243A-489B-B0D2-2F36C2A369FE,C932D197-7F43-4BCF-BE0B-FE5F5DBCB9EF,Family and Children,E9F87919-0CD5-4D9D-AAC6-3CBC49132F1D,Family/Divorce/Custody/Visitation/Child Support,C9C3FA40-0D60-49B9-97F1-63AFEFAFA484,2019-06-20 02:33:14,FA34142B-1575-4720-981C-2D28C3560137,2019-07-27 18:04:08,FA34142B-1575-4720-981C-2D28C3560137,2019-07-31 05:49:18,NaN,215869,NC,"""Due to my husband physically and finance abus...","""2019-06-20 02:33:39"""
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
405510,202878,NC,DC374484-7C6E-44A3-9DB7-89B8868518F1,E6F4FD7C-FCE9-4763-AADD-0544158DE0BB,Other,26D3CE4B-3540-477E-BA03-78EF65769A12,Other,A40F8543-330C-43A1-97AC-53C01FD4013E,2019-06-19 03:23:46,6E43EC98-8700-4439-98A9-52B663CCE348,2019-06-24 14:47:57,6E43EC98-8700-4439-98A9-52B663CCE348,2019-06-28 21:59:13,2019-07-01 00:00:00,215860,NC,"""I am so sorry you are going through this. Re...","""2019-06-24 14:54:48"""
405511,202878,NC,DC374484-7C6E-44A3-9DB7-89B8868518F1,E6F4FD7C-FCE9-4763-AADD-0544158DE0BB,Other,26D3CE4B-3540-477E-BA03-78EF65769A12,Other,A40F8543-330C-43A1-97AC-53C01FD4013E,2019-06-19 03:23:46,6E43EC98-8700-4439-98A9-52B663CCE348,2019-06-24 14:47:57,6E43EC98-8700-4439-98A9-52B663CCE348,2019-06-28 21:59:13,2019-07-01 00:00:00,215861,NC,"""Thank you so much for responding. I am answe...","""2019-06-24 17:00:49"""
405512,202878,NC,DC374484-7C6E-44A3-9DB7-89B8868518F1,E6F4FD7C-FCE9-4763-AADD-0544158DE0BB,Other,26D3CE4B-3540-477E-BA03-78EF65769A12,Other,A40F8543-330C-43A1-97AC-53C01FD4013E,2019-06-19 03:23:46,6E43EC98-8700-4439-98A9-52B663CCE348,2019-06-24 14:47:57,6E43EC98-8700-4439-98A9-52B663CCE348,2019-06-28 21:59:13,2019-07-01 00:00:00,215862,NC,"""OK Thank you for sending these responses. Fi...","""2019-06-25 01:44:56"""
405513,202878,NC,DC374484-7C6E-44A3-9DB7-89B8868518F1,E6F4FD7C-FCE9-4763-AADD-0544158DE0BB,Other,26D3CE4B-3540-477E-BA03-78EF65769A12,Other,A40F8543-330C-43A1-97AC-53C01FD4013E,2019-06-19 03:23:46,6E43EC98-8700-4439-98A9-52B663CCE348,2019-06-24 14:47:57,6E43EC98-8700-4439-98A9-52B663CCE348,2019-06-28 21:59:13,2019-07-01 00:00:00,215863,NC,"""Your information has helped. Thank you for yo...","""2019-06-28 20:2

In [24]:
questOnlyCat = questFull.drop(["Subcategory", "SubcategoryUno"], axis = 1)
# Convert 'post_text' column to string type
questOnlyCat['PostText'] = questOnlyCat['PostText'].astype(str)

questCatPostOnly = questOnlyCat.groupby(['QuestionUno', 'Category'])['PostText'].apply(' '.join).reset_index()
questCatPostOnly

,QuestionUno,Category,PostText
0,00005375-D055-4DE4-8C37-748DC40ADA6E,Family and Children,"""My ex and I went to mediation at an attorney'..."
1,00008AE7-5B06-454D-BA0F-969CE8809C33,"Work, Employment and Unemployment","""In ###. They put a plate in my knee. Had to h..."
2,00011AA0-DE46-498C-AFF1-CF4E5E3E3066,Other,"""So I am wondering if I can still pursue this ..."
3,00014611-580E-4A22-9A8C-51FDB6B6110F,Family and Children,"""Hello, my name is ### Nun Mawi, mother of one..."
4,00023EBF-097B-4F46-81B3-7A495ECE4BFA,Housing and Homelessness,"""### properties is my landlord. They are doin..."
...,...,...,...
164711,FFFD29D5-2707-4B49-BB8B-907835E1F6A1,Other,"""I own a restaurant and we have a mandatory fa..."
164712,FFFD8941-D9FE-4809-A4DB-A9FC49E29C4C,Consumer Financial Questions,"""I Recieved a notice on my parents door this m..."
164713,FFFD9C6C-8F9A-482C-B4CE-21EB9793CF14,"Work, Employment and Unemployment",""" I started with the Vermilion County Housing ..."
164714,FFFEC03F-68C9-4E55-8B62-CA4C7DA99267,Consumer Financial Questions,"""Is it better to have two ### on your credit s..."


In [25]:
# lb = LabelBinarizer()
# lb.fit(questOnlyCat['Category'])
# y = lb.transform(questOnlyCat['Category'])
# lb.classes_

In [26]:
# postTexts = str(questOnlyCat['PostText'])

# stop_words = stopwords.words('english')

# def remove_stop_words(postText):
#   return ' '.join([word for word in postText.split() if word not in stop_words and len(word) > 1])

In [27]:
questCatPostOnly["PostText"] = questCatPostOnly["PostText"].replace(np.nan, '')

In [28]:
fulltext_list = questCatPostOnly["PostText"].tolist()
fulltext = ' '.join(fulltext_list)

In [29]:
char_remov = ['a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z']
for char in char_remov:
    fulltext = fulltext.replace(char, '')

In [30]:
punctuation = np.unique(np.array([*fulltext]))

In [31]:
print(punctuation)

['\x08' '\t' ' ' '!' '"' '#' '$' '%' '&' "'" '(' ')' '*' '+' ',' '-' '.'
 '/' '0' '1' '2' '3' '4' '5' '6' '7' '8' '9' ':' ';' '<' '=' '>' '?' '@'
 '[' '\\' ']' '^' '_' '`' '{' '|' '}' '~' '\x81' '\x8d' '\x8f' '\x90'
 '\x9d' '\xa0' '¡' '¢' '£' '¤' '¥' '¦' '§' '¨' '©' 'ª' '«' '¬' '\xad' '®'
 '¯' '°' '±' '²' '³' '´' 'µ' '¶' '·' '¸' '¹' 'º' '»' '¼' '½' '¾' '¿' 'Â'
 'Ã' 'Ä' 'Å' 'Æ' 'Ç' 'Ê' 'Ë' 'Ì' 'Í' 'Î' 'Ï' 'Ð' 'Ñ' 'Ö' '×' 'Ø' 'Ù' 'Ú'
 'Û' 'á' 'â' 'ã' 'ä' 'å' 'æ' 'ç' 'è' 'é' 'ê' 'ë' 'ì' 'í' 'î' 'ï' 'ð' 'ô'
 'Œ' 'œ' 'Š' 'š' 'Ÿ' 'Ž' 'ž' 'ƒ' 'ˆ' '˜' '–' '—' '‘' '’' '‚' '“' '”' '„'
 '†' '‡' '•' '…' '‰' '‹' '›' '€' '™']


In [32]:
punctuation = np.delete(punctuation, 2)
punctuation

array(['\x08', '\t', '!', '"', '#', '$', '%', '&', "'", '(', ')', '*',
       '+', ',', '-', '.', '/', '0', '1', '2', '3', '4', '5', '6', '7',
       '8', '9', ':', ';', '<', '=', '>', '?', '@', '[', '\\', ']', '^',
       '_', '`', '{', '|', '}', '~', '\x81', '\x8d', '\x8f', '\x90',
       '\x9d', '\xa0', '¡', '¢', '£', '¤', '¥', '¦', '§', '¨', '©', 'ª',
       '«', '¬', '\xad', '®', '¯', '°', '±', '²', '³', '´', 'µ', '¶', '·',
       '¸', '¹', 'º', '»', '¼', '½', '¾', '¿', 'Â', 'Ã', 'Ä', 'Å', 'Æ',
       'Ç', 'Ê', 'Ë', 'Ì', 'Í', 'Î', 'Ï', 'Ð', 'Ñ', 'Ö', '×', 'Ø', 'Ù',
       'Ú', 'Û', 'á', 'â', 'ã', 'ä', 'å', 'æ', 'ç', 'è', 'é', 'ê', 'ë',
       'ì', 'í', 'î', 'ï', 'ð', 'ô', 'Œ', 'œ', 'Š', 'š', 'Ÿ', 'Ž', 'ž',
       'ƒ', 'ˆ', '˜', '–', '—', '‘', '’', '‚', '“', '”', '„', '†', '‡',
       '•', '…', '‰', '‹', '›', '€', '™'], dtype='<U1')

In [35]:
for char in punctuation:
    questCatPostOnly["PostText"] = questCatPostOnly["PostText"].str.replace(char, '')

C:\Users\surab\AppData\Local\Temp/ipykernel_24192/203168138.py:2: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  questCatPostOnly["PostText"] = questCatPostOnly["PostText"].str.replace(char, '')


In [40]:
questCatPostOnly["PostText"] = questCatPostOnly["PostText"].str.lower()

In [42]:
questCatPostOnly.iloc[10:15]

,QuestionUno,Category,PostText
10,0004108C-4961-49E9-8E08-05A8312DF00A,Consumer Financial Questions,also the contract with the dealer shows i owned
11,000471AE-425D-4EDC-98C1-CC9A2CAAC157,Other,in i left my with my while i returned back ...
12,0004ECEE-87E3-46FA-BBE7-856AB4E5EC63,Consumer Financial Questions,i just received a pursuant to dwhat does this...
13,00051686-9547-4243-BE10-DE21BC2A2982,Housing and Homelessness,i received a lease termination letter from my ...
14,000522B2-BDB0-4CF3-8C81-E5722B6B1C51,Family and Children,helloi have court on rd i have attached the or...


In [39]:
questCatPostOnly.to_csv('questCatPostOnly.csv')

In [43]:
len(questCatPostOnly["QuestionUno"].unique())

164716

In [44]:
new = questCatPostOnly.drop_duplicates(subset = "QuestionUno", keep = "first")
new

,QuestionUno,Category,PostText
0,00005375-D055-4DE4-8C37-748DC40ADA6E,Family and Children,my ex and i went to mediation at an attorneys ...
1,00008AE7-5B06-454D-BA0F-969CE8809C33,"Work, Employment and Unemployment",in they put a plate in my knee had to have an...
2,00011AA0-DE46-498C-AFF1-CF4E5E3E3066,Other,so i am wondering if i can still pursue this c...
3,00014611-580E-4A22-9A8C-51FDB6B6110F,Family and Children,hello my name is nun mawi mother of one year ...
4,00023EBF-097B-4F46-81B3-7A495ECE4BFA,Housing and Homelessness,properties is my landlord they are doing my ...
...,...,...,...
164711,FFFD29D5-2707-4B49-BB8B-907835E1F6A1,Other,i own a restaurant and we have a mandatory fac...
164712,FFFD8941-D9FE-4809-A4DB-A9FC49E29C4C,Consumer Financial Questions,i recieved a notice on my parents door this mo...
164713,FFFD9C6C-8F9A-482C-B4CE-21EB9793CF14,"Work, Employment and Unemployment",i started with the vermilion county housing a...
164714,FFFEC03F-68C9-4E55-8B62-CA4C7DA99267,Consumer Financial Questions,is it better to have two on your credit score...
